# Generate and save ECODE responses, protocols, and features

In [1]:
import json
import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import model
import evaluator
import time
import neuron
import plotting
import MEAutility as mu
from pprint import pprint
import numpy as np

import sys
import shutil

from pathlib import Path
import os

# %matplotlib notebook

## 1) Generate and run ecode protocols

In [2]:
sys.path.append('../ecode/')

In [3]:
from ecode import generate_ecode_protocols, compute_rheobase_for_model, run_ecode_protocols, \
        save_intracellular_responses, save_extracellular_template

In [4]:
probe_type = 'linear' #"planar"
model_name = "hay_ais" #"hallermann" #'hay'
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [5]:
# output folder for data
output_folder = Path(f"../data/{model_name}_ecode_probe_{probe_type}")

In [6]:
# compile model-specific mechanisms
if not (model_folder / "x86_64").is_dir():
    curr_dir = Path(".").absolute()
    os.chdir(model_folder)
    print(os.getcwd())
    os.system("nrnivmodl mechanisms")
    os.chdir(curr_dir)
else:
    print(f"Mechanisms already compiled for {model_name}")
    compiled_folder = Path("./x86_64")
    if compiled_folder.is_dir():
        shutil.rmtree(compiled_folder)
    shutil.copytree(model_folder / "x86_64", compiled_folder)

Mechanisms already compiled for hay_ais


In [7]:
cell = model.create(model_name=model_name, release=True)

probe = model.define_electrode(probe_type=probe_type)
probe = None

param_names = [param.name for param in cell.params.values() if not param.frozen]
if model_name == "hallermann":
    cvode_active = False
else:
    cvode_active = True
sim = ephys.simulators.LFPySimulator(cell, cvode_active=cvode_active, electrode=probe)

In [8]:
help(compute_rheobase_for_model)

Help on function compute_rheobase_for_model in module ecode:

compute_rheobase_for_model(cell, sim, step_duration=270, delay=250, step_min=0.1, step_max=1, step_increment=0.02)
    Parameters
    ----------
    cell: BluePyOpt.ephys.Cell
        The BluePyOpt cell model 
    step_duration: float
        Step duration in ms
    delay: float
        Delay in ms before and after the step
    step_min: float
        Minimum step current in nA
    step_max: float
        Maximum step current in nA
    step_increment: float
        Step increment in nA
    
    Returns
    -------
    rheobase_current: float or None
        The estimated rheobase current in nA. If no spikes are found, it returns None



In [9]:
# rheobase, rheo_protocols, rheo_responses = compute_rheobase_for_model(cell, sim=sim, step_min=5.34, step_max=6, step_increment=0.01)
rheobase, rheo_protocols, rheo_responses = compute_rheobase_for_model(cell, sim=sim, step_min=0.32, step_increment=0.005)

Running protocol StepsRheobase_0.32nA
Number of spikes: 0
Running protocol StepsRheobase_0.325nA
Number of spikes: 0
Running protocol StepsRheobase_0.33nA
Number of spikes: 0
Running protocol StepsRheobase_0.335nA
Number of spikes: 0
Running protocol StepsRheobase_0.34nA
Number of spikes: 0
Running protocol StepsRheobase_0.345nA
Number of spikes: 1
Rheobase found


In [10]:
ecode_protocols = generate_ecode_protocols(rheobase_current=rheobase, record_extra=False,
                                           protocols_with_lfp="firepattern")

In [11]:
responses_dict = run_ecode_protocols(protocols=ecode_protocols, cell=cell, sim=sim, 
                                     resample_rate_khz=40)

Running protocol IDthres
Elapsed time IDthres: 61.28082489967346
Running protocol firepattern
Elapsed time firepattern: 33.72769522666931
Running protocol IV
Elapsed time IV: 32.31355953216553
Running protocol IDrest
Elapsed time IDrest: 82.75671052932739
Running protocol APWaveform
Elapsed time APWaveform: 16.412774085998535
Running protocol HyperDepol
Elapsed time HyperDepol: 11.611633062362671
Running protocol sAHP
Elapsed time sAHP: 18.063036680221558
Running protocol PosCheops
Elapsed time PosCheops: 48.20797419548035


In [12]:
save_intracellular_responses(responses_dict=responses_dict, output_folder=output_folder)

Saving IDthres
Saving firepattern
Saving IV
Saving IDrest
Saving APWaveform
Saving HyperDepol
Saving sAHP
Saving PosCheops


In [13]:
eap, locations = save_extracellular_template(responses=responses_dict["firepattern"], 
                                             protocols=ecode_protocols, protocol_name="firepattern",
                                             probe=probe, output_folder=output_folder, sweep_id=1, 
                                             resample_rate_khz=20, fcut=[300, 6000],
                                             filt_type="filtfilt")

AssertionError: firepattern-1.MEA.LFP not found in responses

In [ ]:
ax = mu.plot_mea_recording(eap, probe)

## 2) Generate feature and protocols with BluePyEfe

In [ ]:
sys.path.append('../efeatures_extraction')

In [ ]:
efeatures_output_directory = Path(f"../data/{model_name}_ecode_probe_{probe_type}/efeatures")

In [ ]:
from bluepyefe.extract import read_recordings, extract_efeatures_at_targets, compute_rheobase,\
    mean_efeatures, create_feature_protocol_files
from bluepyefe.plotting import plot_all_recordings_efeatures

from extraction_tools import build_model_metadata, model_csv_reader, get_targets, ecodes_model_timings

In [ ]:
files_metadata = build_model_metadata(cell_id=model_name, ephys_dir=output_folder)
pprint(files_metadata[model_name])

In [ ]:
cells = read_recordings(
    files_metadata=files_metadata,
    recording_reader=model_csv_reader
)

In [ ]:
# define target features for different protocols
targets = get_targets(ecodes_model_timings)
pprint(targets)

In [ ]:
t_start = time.time()
extract_efeatures_at_targets(
    cells, 
    targets,
)
t_stop = time.time()
print(f"Elapsed time {t_stop - t_start}")

In [ ]:
compute_rheobase(
    cells, 
    protocols_rheobase=['IDthres']
)

In [ ]:
protocols = mean_efeatures(cells, targets, use_global_rheobase=True)

In [ ]:
efeatures, protocol_definitions, current = create_feature_protocol_files(
    cells,
    protocols,
    output_directory=efeatures_output_directory,
    threshold_nvalue_save=1,
    write_files=True,
)

## 3) Convert to BPO format and append extra features

In [ ]:
from extraction_tools import convert_to_bpo_format, append_extrafeatures_to_json, compute_extra_features

In [ ]:
protocols_of_interest = ["firepattern_200", "IV_-100", "APWaveform_260"]

in_protocol_path = efeatures_output_directory / "protocols.json"
in_efeatures_path = efeatures_output_directory / "features.json"

out_protocol_path = efeatures_output_directory / "protocols_BPO_test.json"
out_efeatures_path = efeatures_output_directory / "features_BPO_test.json"

In [ ]:
protocols_dict, efeatures_dict = convert_to_bpo_format(in_protocol_path, in_efeatures_path, 
                                                       out_protocol_path, out_efeatures_path, 
                                                       protocols_of_interest=protocols_of_interest, 
                                                       std_from_mean=0.2)

In [ ]:
efeatures_dict

In [ ]:
# append MEA.LFP features
eap = np.load(output_folder / "extracellular" / "template.npy")
fs = np.load(output_folder / "extracellular" / "fs.npy")

In [ ]:
extra_features = compute_extra_features(eap, fs, upsample=10)

In [ ]:
pprint(extra_features)

In [ ]:
efeatures_dict = append_extrafeatures_to_json(extra_features, protocol_name="firepattern_200",
                                              efeatures_path=out_efeatures_path)

In [ ]:
# plot one extra features
plotting.plot_feature_map(extra_features["neg_image"], probe)